### function used to get cutouts and jpegs directly from viewer

In [13]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import glob
import os
import scipy
from astropy.io import fits
import importlib.util
import matplotlib.pyplot as plt
from sklearn import metrics
import matplotlib.backends.backend_pdf
import re
import copy
from astropy.visualization import make_lupton_rgb
import wget
import urllib.request
import shutil

In [21]:
def indices_to_fits(indices, fits_path, new_folder, new_name):
    '''
    Use to generate fits folders from numpy indices, given corresponding parent fits folder. New file is generated
    under new_folder/new_name_{npy idx}.fits
    '''
    for idx in indices[:500]:
        shutil.copy2(fits_path+'cutout_{:06d}.fits'.format(int(idx)), new_folder+new_name+'_{:06d}.fits'.format(int(idx)))

In [18]:
selected_ims = np.load('/Users/mac/Desktop/LBNL/DR7/selected_decals_nonlens_id.npy')
fits_path = '/Users/mac/Desktop/LBNL/DR7/fits_cutouts_dr7/'
new_folder = '/Users/mac/Desktop/LBNL/DR7/selected_decals_nonlenses_fits/'
new_name = 'selected_decals_nonlens'
indices_to_fits(selected_ims, fits_path, new_folder, new_name)

In [22]:
indices = np.load('/Users/mac/Desktop/LBNL/DR7/kept_decals_nonlenses.npy')
fits_path = '/Users/mac/Desktop/LBNL/DR7/fits_cutouts_dr7/'
new_folder = '/Users/mac/Desktop/LBNL/DR7/kept_decals_nonlenses_check_fits/'
new_name = 'kept_decals_nonlens_check'
indices_to_fits(selected_ims, fits_path, new_folder, new_name)

In [66]:
def download_jpegs_fits(fits_folder, jpeg_folder, dictionary_path, save_idx=0, layer='decals-dr7'):
    '''
    * NOT QUITE FINISHED FOR FULL OPTIMIZATION * a bit hardcoded still
    enter ra+dec from legacy survey viewer as in format: 24.3948&dec=-13.0079
    
    function splits up ra and dec, downloads jpeg and fit cutout centered at that location. need to specify
    folders, layer.
    
    this is so that we can manually look at the skyviewer and easily enter coordinates of objects we want cutouts of.
    
    
    '''
    cont = True
    while cont:
        # get user inputs (ra dec cluster)
        ra_dec = input('enter ra, dec: ')
        if ra_dec == 'exit' or ra_dec == 'quit':
            print('*user exit')
            cont = False
            break
        ra, dec = ra_dec.split('&dec=')
        cluster = input('enter cluster name: ')
        if cluster == 'exit' or cluster == 'quit':
            print('*user exit')
            cont = False
            break
            
        # download jpeg
        urllib.request.urlretrieve('http://legacysurvey.org/viewer/jpeg-cutout?ra={}&dec={}&layer={}&pixscale=0.262&bands=grz&size=101'.format(ra, dec, layer), jpeg_folder+'cluster_nonlens_{:06d}.jpeg'.format(save_idx))
        
        # download fits
        url = 'http://legacysurvey.org/viewer/fits-cutout?ra={}&dec={}&layer={}&pixscale=0.262&bands=grz&size=101'.format(ra, dec, layer)
        filename = wget.download(url, fits_folder+'cluster_nonlens_{:06d}.fits'.format(save_idx))

        # append id, ra, dec, cluster name to dictionary array
        dict_arr = np.load(dictionary_path)
        dict_arr[save_idx] = {'cluster_nonlens_id':'cluster_nonlens_{:06d}'.format(save_idx), 'ra':ra, 'dec':dec, 'cluster':cluster}
        np.save(dictionary_path,dict_arr)
        save_idx += 1
        
    print('exiting. stopped on idx ', save_idx)
    return save_idx

In [21]:
# testing different splits
ra_dec = input('enter ra, dec: ')
ra, dec = ra_dec.split(', ')
print(ra, dec)

enter ra, dec: 200.1760, 32.0599
200.1760 32.0599


In [74]:
# START ON IDX 7
save_idx = download_jpegs_fits(fits_folder, jpeg_folder, dictionary_path, save_idx, layer='decals-dr7')

enter ra, dec: 24.3948&dec=-13.0080
enter cluster name: abell 222
enter ra, dec: 24.3597&dec=-12.9993
enter cluster name: abell 222
enter ra, dec: 24.3948&dec=-13.0079
enter cluster name: abell 222
enter ra, dec: exit
*user exit
exiting. stopped on idx  7


In [75]:
np_dict = np.load(dictionary_path)
np_dict[:10]

array([ {'cluster_nonlens_id': 'cluster_nonlens_000000', 'ra': '24.3889', 'dec': '-12.989', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'cluster_nonlens_000001', 'ra': '24.3895', 'dec': '-12.9935', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'cluster_nonlens_000002', 'ra': '24.3762', 'dec': '-12.9945', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'cluster_nonlens_000003', 'ra': '24.3889', 'dec': '-12.9860', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'cluster_nonlens_000004', 'ra': '24.3948', 'dec': '-13.0080', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'cluster_nonlens_000005', 'ra': '24.3597', 'dec': '-12.9993', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'cluster_nonlens_000006', 'ra': '24.3948', 'dec': '-13.0079', 'cluster': 'abell 222'},
       {'cluster_nonlens_id': 'none', 'ra': 0, 'dec': 0, 'cluster': 'none'},
       {'cluster_nonlens_id': 'none', 'ra': 0, 'dec': 0, 'cluster': 'none'},
       {'cluster_nonlens_id